In [1]:
%run "Properati_base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
chart-studio=1.1.0 already installed
nodejs=13.13.0 already installed
psutil=5.7.0 already installed
plotly-orca=1.3.1 already installed


In [2]:
import re 
import numpy as np 
import pandas as pd

In [3]:
data = pd.read_csv("properati.csv", sep=",")
#data.head(3)

In [4]:
print("cantidad de filas: " + str(data.shape[0]))
print("cantidad de columnas: " + str(data.shape[1]))

cantidad de filas: 121220
cantidad de columnas: 26


In [5]:
## [BORRAR_PRESENCIAL]
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [6]:
#TABLA NULOS POR COLUMNA

data_null_pct = round(data.isnull().sum() / data.shape[0] * 100, 2).to_frame()
data_null_count = data.isnull().sum().to_frame()
#data_null_table = data_null_count.join(data_null_avg)
#data_null_table
data_null_count.rename(columns={0:'valores'}, inplace=True)
data_null_pct.rename(columns={0:'pct'}, inplace=True)
data_null_table = data_null_count.join(data_null_pct)
data_null_table

,valores,pct
Unnamed: 0,0,0.00
operation,0,0.00
property_type,0,0.00
place_name,23,0.02
place_with_parent_names,0,0.00
country_name,0,0.00
state_name,0,0.00
geonames_id,18717,15.44
lat-lon,51550,42.53
lat,51550,42.53


In [7]:
data_precios_m2 = data.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2','expenses','surface_total_in_m2','surface_covered_in_m2']]
data_precios_m2

,price,currency,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,expenses,surface_total_in_m2,surface_covered_in_m2
0,62000.0,USD,1093959.00,62000.0,1127.272727,1550.000000,NaN,55.0,40.0
1,150000.0,USD,2646675.00,150000.0,NaN,NaN,NaN,NaN,NaN
2,72000.0,USD,1270404.00,72000.0,1309.090909,1309.090909,NaN,55.0,55.0
3,95000.0,USD,1676227.50,95000.0,NaN,NaN,NaN,NaN,NaN
4,64000.0,USD,1129248.00,64000.0,1828.571429,1828.571429,NaN,35.0,35.0
...,...,...,...,...,...,...,...,...,...
121215,870000.0,USD,15350715.00,870000.0,7699.115044,9354.838710,10000.0,113.0,93.0
121216,498000.0,USD,8786961.00,498000.0,1383.333333,1383.333333,NaN,360.0,360.0
121217,131500.0,USD,2320251.75,131500.0,2858.695652,3371.794872,NaN,46.0,39.0
121218,95900.0,USD,1692107.55,95900.0,1997.916667,1997.916667,NaN,48.0,48.0


In [8]:
#Analisis de nº piso
data_precios_m2_floor = data.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2','expenses','surface_total_in_m2','surface_covered_in_m2', 'floor']]
floor_not_null_mask = data_precios_m2_floor.floor.notnull()
data_precios_m2_floor_grouped = data_precios_m2_floor.loc[floor_not_null_mask,:].groupby('floor').size()
#display(data_precios_m2_floor_grouped)
data_floor_over_mask = data_precios_m2_floor_grouped.index > 1000
data_floor_over_value = data_precios_m2_floor_grouped.loc[data_floor_over_mask]
data_floor_over_value

floor
1001.0    1
1104.0    2
1106.0    1
1109.0    1
1201.0    1
1202.0    1
1204.0    1
1206.0    1
1208.0    1
1291.0    1
1301.0    1
1304.0    1
1305.0    1
1306.0    1
1379.0    1
1401.0    1
1403.0    1
1404.0    1
1407.0    1
1420.0    1
1450.0    1
1504.0    2
1512.0    1
1602.0    1
1606.0    1
1607.0    1
1608.0    2
1609.0    4
2103.0    1
2177.0    1
2509.0    1
3150.0    1
dtype: int64

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(data_floor_over_value, kde=False);
#plt.axvline(10, color="k", linestyle="--");

In [10]:
data_floor_notnull_avobe =  data_precios_m2_floor.floor < 6
data_precios_m2_floor['USD_range'] = pd.qcut(data_precios_m2_floor.price_aprox_usd, 10)
data_precios_m2_floor.groupby('USD_range').size()

USD_range
(-0.001, 65000.0]         10695
(65000.0, 81600.0]         9469
(81600.0, 98146.43]       10150
(98146.43, 120000.0]      10593
(120000.0, 145000.0]      10031
(145000.0, 179000.0]       9724
(179000.0, 231027.3]       9905
(231027.3, 310000.0]      10142
(310000.0, 470000.0]      10090
(470000.0, 46545445.0]    10011
dtype: int64

In [11]:
#Analisis de nº piso
data_precios_m2_rooms = data.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2','expenses','surface_total_in_m2','surface_covered_in_m2', 'rooms']]
data_precios_m2_rooms['USD_range'] = pd.qcut(data_precios_m2_floor.price_aprox_usd, 10)
rooms_not_null_mask = data_precios_m2_rooms.rooms.notnull()
data_precios_m2_rooms_grouped = data_precios_m2_rooms.loc[rooms_not_null_mask,:].groupby(['rooms', 'USD_range']).size().to_frame()
#display(data_precios_m2_rooms_grouped)
table = pd.pivot_table(data_precios_m2_rooms_grouped, index=['rooms'], columns=['USD_range'])
df = data_precios_m2_rooms_grouped.reset_index()
df

,rooms,USD_range,0
0,1.0,"(-0.001, 65000.0]",2029
1,1.0,"(65000.0, 81600.0]",1036
2,1.0,"(81600.0, 98146.43]",1014
3,1.0,"(98146.43, 120000.0]",831
4,1.0,"(120000.0, 145000.0]",366
...,...,...,...
305,32.0,"(145000.0, 179000.0]",0
306,32.0,"(179000.0, 231027.3]",0
307,32.0,"(231027.3, 310000.0]",1
308,32.0,"(310000.0, 470000.0]",0


In [12]:
"""from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

data_bokeh = ColumnDataSource(data)

options = {'plot_width': 900,
           'plot_height': 300,
           'tools': 'pan, box_select, lasso_select, reset'}

p1 = figure(title="Precio x cant_hab", x_axis_label = "H", y_axis_label = "P", **options)
p1.circle("rooms", "price_aprox_usd", color = "blue", source = data_bokeh)

output_notebook()
show(p1)"""

'from bokeh.plotting import figure, output_notebook, show\nfrom bokeh.models import ColumnDataSource\nfrom bokeh.layouts import gridplot\n\ndata_bokeh = ColumnDataSource(data)\n\noptions = {\'plot_width\': 900,\n           \'plot_height\': 300,\n           \'tools\': \'pan, box_select, lasso_select, reset\'}\n\np1 = figure(title="Precio x cant_hab", x_axis_label = "H", y_axis_label = "P", **options)\np1.circle("rooms", "price_aprox_usd", color = "blue", source = data_bokeh)\n\noutput_notebook()\nshow(p1)'

In [13]:
#Ver cuantos valores de currency tienen valor distinto de 'USD'
currency_mask = (data.currency != 'USD') & (data.currency.notnull()) &  (data.currency != 'ARS')
print('Cant registros en no Nulos, no USD y no ARS: ', currency_mask.sum())
data_currency_not_null_USD_ARS = data_precios_m2.loc[currency_mask]
data_currency_not_null_USD_ARS

Cant registros en no Nulos, no USD y no ARS:  3


,price,currency,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,expenses,surface_total_in_m2,surface_covered_in_m2
50387,380000.0,PEN,2.066865e+06,117139.33,NaN,NaN,NaN,NaN,NaN
50388,950000.0,PEN,5.167162e+06,292848.33,NaN,NaN,NaN,NaN,NaN
107390,650000000.0,UYU,4.054773e+08,22980378.29,NaN,NaN,NaN,NaN,NaN


In [14]:
#Ver cuantos valores de observaciones tienen valor distinto a sell
rent_mask = data.operation != 'sell'
rent_mask.sum()
#data.loc[rent_mask,["operation", "price"]].sample(30)

0

In [15]:
place_with_parent_names_GROUPED = data.groupby('place_with_parent_names').size().sort_values(ascending=False)
place_with_parent_names_GROUPED.head(30)

place_with_parent_names
|Argentina|Santa Fe|Rosario|                                          8504
|Argentina|Córdoba|Córdoba|                                           6606
|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|                6534
|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nordelta|                   3315
|Argentina|Capital Federal|Belgrano|                                  2992
|Argentina|Capital Federal|Palermo|                                   2885
|Argentina|Córdoba|                                                   2648
|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|                            2359
|Argentina|Capital Federal|Caballito|                                 2273
|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|                            1857
|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|La Plata|                  1767
|Argentina|Capital Federal|Villa Urquiza|                             1629
|Argentina|Bs.As. G.B.A. Zona Norte|Vicente López|Olivos|             1590
|

In [16]:
import re

pattern = 'Argentina'
pattern_regex = re.compile(pattern)
result = place_with_parent_names_GROUPED.index.to_series().apply(lambda x: pattern_regex.search(x))
result.notnull().sum()

1164

In [17]:
place_with_parent_names_GROUPED.index

Index(['|Argentina|Santa Fe|Rosario|', '|Argentina|Córdoba|Córdoba|',
       '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nordelta|',
       '|Argentina|Capital Federal|Belgrano|',
       '|Argentina|Capital Federal|Palermo|', '|Argentina|Córdoba|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|',
       '|Argentina|Capital Federal|Caballito|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|',
       ...
       '|Argentina|Buenos Aires Interior|Pergamino|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Barrio Cerrado "Isla del Sol"|',
       '|Argentina|Buenos Aires Interior|Navarro|Navarro|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Barrio Cerrado "La Escondida"|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Barrio Cerrado Lagos del Norte|',
       '|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Barrio Cerrado San Andres|',
       '|Argentina|Buenos Aires Interior|Olavarría|Sierras Bayas|',
       '|Ar

In [ ]:
import re
import pandas as pd
pattern = r'http:\/\/www\.properati.com.ar\/\w{6}([\w*-]*)'
pattern_regex = re.compile(pattern)
result_match = data['properati_url'].apply(lambda x: pattern_regex.match(x.replace('-','_')))
result =  result_match.apply(lambda x: x.group(1).split('_'))

list_words = []

def generate_words_series(arr):
    for x in arr:
        list_words.append(str(x))
    return

result.apply(generate_words_series)

serie = pd.Series(list_words)
key_words = serie.value_counts()



In [19]:
display(key_words.head(40))


venta            121119
departamento      71019
garage            56620
lavadero          44034
balcon            42288
propiedades       41507
casa              40537
parrilla          33381
piscina           32148
luminoso          31865
suite             27102
placard           26145
terraza           25101
toilette          22841
subte             21984
linea             21824
patio             21169
vestidor          18011
jardin            17173
quincho           12850
aire              12767
acondicionado     12763
inmobiliaria      12184
sum               11665
dependencias      11316
max               11260
re                11258
cordoba           11195
amenities         10607
baulera           10512
estrenar          10383
vista             10309
de                 9768
san                9556
gimnasio           9363
lujoso             9251
plata              9215
del                9180
rosario            8607
villa              8473
dtype: int64

<h1>Observaciones</h1>
<ul>
    <li>El campo 'operation' solo tiene valor 'sell' </li>
    <li>place_with_parent_names tiene 1164 valores y todos contienen Argentina </li>
    <li>Aparentemente es una inmobiliaria de Rosario que se expandio al país</li>
    <li>existen 2 registros con tipo de moneda PEN y uno con tipo de moneda UYU, los demas son USD y ARS</li>
    <!--<li></li>-->
</ul>

<h1>Todo:</h1>
<ul>
    <li>Completar valores de expensas de departamentos</li>
    <li>Generar un analisis de palabras claves en descripción</li>
    <li>Price, currency, price_aprox_local_currency, price_aprox_usd completar al fonal en base al modelo generado con todas las variables</li>
    <li>Crear columna valor en USD de todos los que tienen valor ARS</li>
    <li>Crear tabla con descripcion de valores en dolares segun cantidad de ambientes segmentados por m2</li>
    <li>Encontrar outlyears en m2, precios, ambientes</li>
</ul>
